# split to passages

In [3]:
from dataclasses import dataclass
import PyPDF2
import re

def read_pdf_pages(file_path):
    text = []
    with open(file_path, 'rb') as f:
        pdfReader = PyPDF2.PdfReader(f)
        count = len(pdfReader.pages)
        for i in range(count):
            page = pdfReader.pages[i]
            output = page.extract_text()
            text.append(output)
            text.append(f'PAGE={i + 1}')
    return text


def parse_passage_num(line):
    num = -1
    gr1 = re.search(r'^\s*(Раздел\s+\d+(\.\d+)*\.?|(\d+\.\d+(\.\d+)*))\.?', line)
    if gr1:
        num = gr1.group(0).strip()
        if re.match(r'^\d{1,2}$', num) or re.search(r'^\d{2}\.\d{2}\.\d{4}', line.strip()):
            num = -1

    return num

@dataclass
class Passage:
    text: str = ''
    passage_number: str = None
    page_number_first: int = None
    page_number_last: int = None
    text_with_meta: str = ''


class NPA2:
    def __init__(self, file_path, passage_len_limits=[10, 80000]):
        self.doc_text = read_pdf_pages(file_path)
        self.doc_name = file_path.split('/')[-1]
        self.passage_len_limits = passage_len_limits
        self.preproc_npa_text()
        self.extract_passages()
        self.add_meta_info_passages()

    def preproc_npa_text(self):
        text_plain = '\n'.join(self.doc_text)
        text_plain = re.sub(r'\n ', '\n', text_plain)
        text_plain = text_plain.replace('www.consultant.ru', '')
        self.doc_text = text_plain

    def extract_passages(self):
        lines = self.doc_text.split('\n')
        passages = []
        cur_passage = Passage()
        page_num = 1

        for line in lines:
            if 'PAGE=' in line:
                page_num = line.split('PAGE=')[-1]
            line_text = line.strip()
            num = parse_passage_num(line_text)
            
            if num != -1:
                if cur_passage.text:
                    passages.append(cur_passage)
                cur_passage = Passage(text=line_text, passage_number=num,
                                      page_number_first=page_num, page_number_last=page_num)
            else:
                if cur_passage.passage_number is not None:
                    if cur_passage.text or line_text:
                        cur_passage.text += '\n' + line_text
                        cur_passage.page_number_last = page_num

        if cur_passage.text:
            passages.append(cur_passage)

        for p in passages:
            p.text = re.sub(r'PAGE=\d+', '', p.text)
            p.text = re.sub(r'(\D)\n+(\D)', r'\1 \2', p.text.strip())

        self.passages = [p for p in passages if self.passage_len_limits[0] <= len(p.text) <= self.passage_len_limits[1]]

    def add_meta_info_passages(self):
        level_texts = [None] * 10  

        for p in self.passages:
            if p.passage_number.startswith('Раздел'):
                level_texts[0] = p.text  
                p.text_with_meta = p.text
            else:
                levels = p.passage_number.split('.')

                for i, level in enumerate(levels):
                    level_texts[i + 1] = level_texts[i + 1] if level_texts[i + 1] else ""
                    if i == len(levels) - 1: 
                        level_texts[i + 1] = p.text

                p.text_with_meta = '\n'.join(filter(None, level_texts[:len(levels) + 1]))


In [4]:
passages_df_full = []
file_names = ['Коллективный договор.pdf']
data_path_npa_pdf = './'

for name in file_names:
    passages_npa_df = []
    npa = NPA2(data_path_npa_pdf + name)

    for p in npa.passages:
        passages_npa_df.append(
            {
                'passage': p.text,
                'passage_full': p.text_with_meta,
                'number': p.passage_number,
                'page_first': p.page_number_first,
                'page_last': p.page_number_last,
                'title': name,
            }
        )
    passages_df_full.extend(passages_npa_df)
    print(passages_npa_df)
    print(f'add {name} to dataframe')

[{'passage': 'Раздел 1. Основные понятия Коллективный договор ОАО «РЖД» на 2023 - 2025 годы - правовой акт, регулирующий социально -трудовые отношения в открытом акционерном обществе «Российские железные дороги»  между сторонами социального партнерства - Работниками и Работодателем в лице их представителей. Настоящий Договор является единым для ОАО «РЖД», включая филиалы, структурные подразделения и представительства, за исключением Петропавловского отделения Южно -Уральской железной дороги - филиала ОАО «РЖД», расположенного на территории Республики  Казахстан, в котором, на основе настоящего Договора и с учетом особенностей законодательства Республики Казахстан, заключается отдельный коллективный договор на 2023 -\n2025 годы. Предоставление Работникам одних и тех же гарантий, льгот и компенсаций в соответствии с Договором и коллективным договором указанного филиала не допускается. Нормы раздела 9  настоящего Договора не применяются в отношении указанного филиала. При этом, нормы, рег

In [5]:
import pandas as pd
df = pd.DataFrame(passages_df_full)

In [6]:
df.tail(50)

,passage,passage_full,number,page_first,page_last,title
180,9.17. Выплачивать председателям первичных проф...,Раздел 9. Обязательства в сфере социального па...,9.17.,45,45,Коллективный договор.pdf
181,9.18. Производить по письменному заявлению Р...,Раздел 9. Обязательства в сфере социального па...,9.18.,45,45,Коллективный договор.pdf
182,9.19. Возмещать расходы Профсоюза на предостав...,Раздел 9. Обязательства в сфере социального па...,9.19.,45,45,Коллективный договор.pdf
183,7.1. - 7.12. настоящего Договора. Возмещать ра...,Раздел 9. Обязательства в сфере социального па...,7.1.,45,46,Коллективный договор.pdf
184,9.20. Привлекать к дисциплинарной ответстве нн...,Раздел 9. Обязательства в сфере социального па...,9.20.,46,46,Коллективный договор.pdf
185,9.21. Предоставлять Работникам - общественным ...,Раздел 9. Обязательства в сфере социального па...,9.21.,46,46,Коллективный договор.pdf
186,9.22. Приглашать представителей Профсоюза на з...,Раздел 9. Обязательства в сфере социального па...,9.22.,46,46,Коллективный договор.pdf
187,Раздел 10. Обязательства Работнико в,Раздел 10. Обязательства Работнико в,Раздел 10.,46,46,Коллективный договор.pdf
188,10.1. Соблюдать Правила внутреннего трудового ...,Раздел 10. Обязательства Работнико в\n10.1. Со...,10.1.,46,46,Коллективный договор.pdf
189,"10.2. Способствовать внедре нию инноваций, пос...",Раздел 10. Обязательства Работнико в\n10.2. Сп...,10.2.,46,46,Коллективный договор.pdf


In [7]:
df.values[14]

array(['3.2.2. сохранять за семьями Работников, погибших или получивших\n1 группу инвалидности в результате несчастного случая на производстве, право на корпоративную по ддержку при строительстве (приобретении) жилья в собственность, а также право пользоваться образовательными учреждениями, учрежденными Компанией, путевками в детские оздоровительные лагеря в пределах бюджетных параметров на условиях, установленных Компание й с учетом мотивированного мнения выборного органа Профсоюза;',
       'Раздел 3. Социальная ответственность Компании\n3.2. Работодатель обязуетс я:\n3.2.2. сохранять за семьями Работников, погибших или получивших\n1 группу инвалидности в результате несчастного случая на производстве, право на корпоративную по ддержку при строительстве (приобретении) жилья в собственность, а также право пользоваться образовательными учреждениями, учрежденными Компанией, путевками в детские оздоровительные лагеря в пределах бюджетных параметров на условиях, установленных Компание й с 

In [ ]:
df.to_pickle('passages_from_doc_upd.pkl')